### Problem (Business Case)

**Description**: "Healthcare for All", is a not-for-profit organization that provides financial help to people who are not able to afford healthcare. This organization raises funds through donations from all across the country. 

One of their most efficient channels for a long period of time to reach out to donors has been direct mails, but for the last couple of years, they have seen a decline in the donations through this medium.

One group that is of particular interest to the organization is "Lapsed" donors. These are individuals who made their last donation 13 to 24 months ago. They represent an important group to the organization, since the longer someone goes without donating, the less likely they will be to give again. Therefore, the recapture of these former donors is a critical aspect of their fund raising efforts.

However, it was found that there is often an inverse correlation between likelihood to respond and the dollar amount of the gift, so a straight response model (a classification or discrimination task) will most likely net only very low dollar donors. High dollar donors will fall into the lower deciles, which would most likely be suppressed from future mailings. The lost revenue of these suppressed donors would then offset any gains due to the increased response rate of the low dollar


**Goal**: Improve the cost-effectiveness of future direct marketing efforts of "Healthcare for All" organization by developing a model that will help them to maximize the net revenue (a regression or estimation task) generated from future renewal mailings to Lapsed donors. Predict higher donation donors.

**Population for this analysis**: Lapsed donors who received the June 1997 renewal mailing

TARGET_B: Responders to mailing. If responded or not.

Target_D: Total dollar amount donated by each responder.

In [1]:
import pandas as pd
import numpy as np
import scipy as stats

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import datetime 

import warnings
warnings.filterwarnings('ignore')

#### Importing Data

In [2]:
data = pd.read_csv("learningSet.csv")
data.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [3]:
data.shape

#95412 rows, 481 columns

(95412, 481)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Columns: 481 entries, ODATEDW to GEOCODE2
dtypes: float64(97), int64(310), object(74)
memory usage: 350.1+ MB


In [5]:
data.describe()

,ODATEDW,TCODE,DOB,AGE,NUMCHLD,INCOME,WEALTH1,HIT,MBCRAFT,MBGARDEN,...,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,TARGET_B,TARGET_D,HPHONE_D,RFA_2F,CLUSTER2
count,95412.000000,95412.000000,95412.000000,71747.000000,12386.000000,74126.000000,50680.000000,95412.000000,42558.000000,42558.000000,...,95412.000000,85439.000000,85439.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95412.000000,95280.000000
mean,9141.363256,54.223117,2723.602933,61.611649,1.527773,3.886248,5.345699,3.321438,0.152075,0.059166,...,9135.651648,9151.022917,8.093739,13.347786,95778.176959,0.050759,0.793073,0.500618,1.910053,31.533711
std,343.454752,953.844476,2132.241295,16.664199,0.806861,1.854960,2.742490,9.306899,0.470023,0.262078,...,320.394019,294.257260,8.213242,10.769997,55284.596094,0.219506,4.429725,0.500002,1.072749,18.764614
min,8306.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7211.000000,0.000000,1.285714,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,8801.000000,0.000000,201.000000,48.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,...,8810.000000,8903.000000,4.000000,8.384615,47910.750000,0.000000,0.000000,0.000000,1.000000,15.000000
50%,9201.000000,1.000000,2610.000000,62.000000,1.000000,4.000000,6.000000,0.000000,0.000000,0.000000,...,9201.000000,9204.000000,6.000000,11.636364,95681.500000,0.000000,0.000000,1.000000,2.000000,32.000000
75%,9501.000000,2.000000,4601.000000,75.000000,2.000000,5.000000,8.000000,3.000000,0.000000,0.000000,...,9409.000000,9409.000000,11.000000,15.477955,143643.500000,0.000000,0.000000,1.000000,3.000000,49.000000
max,9701.000000,72002.000000,9710.000000,98.000000,7.000000,7.000000,9.000000,241.000000,6.000000,4.000000,...,9603.000000,9702.000000,1088.000000,1000.000000,191779.000000,1.000000,200.000000,1.000000,4.000000,62.000000


## 1. Complete the following steps on the categorical columns in the dataset:

#### 1.1 Check for null values in all the columns

In [6]:
categor=data.select_dtypes(np.object)
categor.isna().sum().to_frame().rename(columns={0:'count'}).sort_values(by='count', ascending=False)

#132 null values in column "GEOCODE2"

,count
GEOCODE2,132
RFA_12,0
RFA_10,0
RFA_9,0
RFA_8,0
...,...
GEOCODE,0
MAJOR,0
SOLIH,0
SOLP3,0


#### 1.2 Exclude the following variables by looking at the definitions. Create a new empty list called drop_list. We will append this list and then drop all the columns in this list later:

OSOURCE - symbol definitions not provided, too many categories

ZIP CODE - we are including state already

In [7]:
drop_list=[]
cols=data[["OSOURCE", "ZIP"]]
for i in cols:
    drop_list.append(i)
drop_list


['OSOURCE', 'ZIP']

In [ ]:
#categor.drop(columns=drop_list,axis=1,inplace=True)

In [ ]:
#OR
#drop_list = list(data[["OSOURCE", "ZIP"]])

#### 1.3 Identify columns that over 85% missing values

In [8]:
nulls = pd.DataFrame(categor.isna().sum()*100/len(categor), columns=['percentage'])
nulls.sort_values('percentage', ascending = False).head(10)

# No categorical columns with percentage of missing values above 85%!!

,percentage
GEOCODE2,0.138347
RFA_12,0.000000
RFA_10,0.000000
RFA_9,0.000000
RFA_8,0.000000
RFA_7,0.000000
RFA_6,0.000000
RFA_5,0.000000
RFA_4,0.000000
RFA_3,0.000000


In [36]:
categor.head()

,OSOURCE,STATE,ZIP,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,GRI,IL,61081,,,0,,,,,...,S4E,S4E,S4E,S4E,L,E,X,X,X,C
1,BOA,CA,91326,,,0,,,,,...,N1E,N1E,,F1E,L,G,X,X,X,A
2,AMH,NC,27017,,,0,,,,,...,,S4D,S4D,S3D,L,E,X,X,X,C
3,BRY,CA,95953,,,0,,,,,...,A1D,A1D,,,L,E,X,X,X,C
4,,FL,33176,,,0,X,X,,,...,A3D,I4E,A3D,A3D,L,F,X,X,X,A


HOWEVER, There are a lot of columns with empty values which are not identified as "null values".
Let's replace these empty spaces with NaNs and repeat the analysis, but first let's look at "MAILCODE" Column.

In [41]:
#COLUMN "MAILCODE", empty space means "Address is ok" and B "Bad address" when reading column definition. 
categor['MAILCODE'].value_counts()

     94013
B     1399
Name: MAILCODE, dtype: int64

In [42]:
categor['MAILCODE'].unique()

array([' ', 'B'], dtype=object)

In [43]:
#Let's replace " " with "A" in "MAILCODE" column.

categor['MAILCODE'] = np.where(categor['MAILCODE'] == " ","A",categor['MAILCODE'])

In [44]:
categor.MAILCODE.value_counts()

A    94013
B     1399
Name: MAILCODE, dtype: int64

In [46]:
# replace these empty spaces with NaNs
categor = categor.apply(lambda x: x.replace(" ", np.NaN))

In [54]:
nulls_percent = pd.DataFrame(categor.isna().sum()*100/len(categor)).reset_index() 
nulls_percent.columns = ['column_name', 'nulls_percentage']
nulls_percent.sort_values('nulls_percentage', ascending = False).head(30)

,column_name,nulls_percentage
8,RECPGVG,99.880518
21,SOLP3,99.811344
23,MAJOR,99.691863
42,PLATES,99.413072
29,HOMEE,99.070348
41,CARDS,98.908942
15,CHILD03,98.798893
4,PVASTATE,98.471890
40,KIDSTUFF,98.390140
16,CHILD07,98.358697


#### 1.4 Remove those columns from the dataframe (columns with missing values above 85%)

In [58]:
columns_above_threshold = nulls_percent[nulls_percent['nulls_percentage']>85]
columns_above_threshold.sort_values('nulls_percentage')
drop_columns_list = list(columns_above_threshold['column_name'])
drop_columns_list

['PVASTATE',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES']

In [59]:
categor2 = categor.drop(columns=drop_columns_list)

In [ ]:
# FUNCTION to remove nulls above a specific thershold

#def filter_nulls(data, perc = .85):
  #nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()  
 #nulls_percent_df.columns = ['column_name', 'nulls_percentage']
  #columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>perc]  #ist of columns which have a percentage of null values above the specified thershold.
  #drop_columns_list = list(columns_above_threshold['column_name'])
  #return drop_columns_list
    
    
#drop_columns_list = filter_nulls(data)
#drop_columns_list

#data = data.drop(columns=drop_columns_list)
#data.shape

In [60]:
categor.shape

(95412, 74)

In [61]:
categor2.shape

(95412, 45)

In [62]:
# 29 Columns removed.

#### 1.5 Reduce the number of categories in the column GENDER. 
#### The column should only have either "M" for males, "F" for females, and "other" for all the rest

In [63]:
categor2.GENDER.unique()

array(['F', 'M', nan, 'C', 'U', 'J', 'A'], dtype=object)

In [64]:
categor2.GENDER.value_counts()

F    51277
M    39094
U     1715
J      365
C        2
A        2
Name: GENDER, dtype: int64

In [ ]:
#data2['GENDER'] = data2['GENDER'].apply(lambda x: x.replace(" ", "F"))
#data['GENDER'] = np.where(data['GENDER'] == " ","F",data2['GENDER'])

In [65]:
def gender(val):
    if val == "F":
        return "F"
    elif val == "M":
        return "M"
    elif val == " ":
        return "F"
    else:
        return "other"

In [66]:
categor2['GENDER'] = categor2['GENDER'].apply(gender)

In [67]:
categor2.GENDER.value_counts()

F        51277
M        39094
other     5041
Name: GENDER, dtype: int64